# MNIST with Eager Execution.

* MNIST data converted to `tf.data` [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)
* This enables convenient data manipulation: parallel reads, prefetching, batching, shuffling, etc
* Learn more about `tf.data`: https://www.tensorflow.org/programmers_guide/datasets

Code is similar to the previous notebook with a few modifications:

1. Instead of using `model.fit`, we use `model.train_on_batch` to demonstrate batch iteration over the dataset. 
2. Using TensorFlow optimizer to pass to the Keras model.
3. Using enable eager execution.

## Eager execution
Eager execution is a mode for running TensorFlow that works just like regular Python. https://www.tensorflow.org/programmers_guide/eager



In [1]:
!date
!hostname
!pip install --ignore-installed --upgrade tensorflow==1.8.0

import tensorflow as tf

# Enable eager execution
tf.enable_eager_execution()

import numpy as np

Thu May 17 22:13:20 UTC 2018
smallDLVM1
  Using cached https://files.pythonhosted.org/packages/6d/dc/464f59597a5a8282585238e6e3a7bb3770c3c1f1dc8ee72bd5be257178ec/tensorflow-1.8.0-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/b2/91/cc9805f1ff7b49f620136b3a7ca26f6a1be2ed424606804b0fbcf499f712/astor-0.6.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5b/c3/9b947e301e19bea75dc8c1fd3710eed5d2b31aa13ae13d5e38e891f784cc/protobuf-3.5.2.post1-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/59/a6/0ae6092b7542cfedba6b2a1c9b8dceaf278238c39484f3ba03b03f07803c/tensorboard-1.8.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/81/30/e935244ca6165187ae8be876b6316ae201b71485538ffac1d718843025a9/wheel-0.31.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/67/4b/141a581104b1f6397bfa78ac9d43d8ad29a7ca43ea90a2d863fe3056e86a/six-1.11.0-py2.py3-non

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [3]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

# Reshape from (N, 28, 28) to (N, 784)
train_images = np.reshape(train_images, (TRAINING_SIZE, 784))
test_images = np.reshape(test_images, (TEST_SIZE, 784))

# Convert the array to float32 as opposed to uint8
train_images = train_images.astype(np.float32)
test_images = test_images.astype(np.float32)

# Convert the pixel values from integers between 0 and 255 to floats between 0 and 1
train_images /= 255
test_images /=  255

In [4]:
NUM_DIGITS = 10

print("Before", train_labels[0]) # The format of the labels before conversion

train_labels  = tf.keras.utils.to_categorical(train_labels, NUM_DIGITS)

print("After", train_labels[0]) # The format of the labels after conversion

test_labels = tf.keras.utils.to_categorical(test_labels, NUM_DIGITS)

Before 5
After [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [5]:
# Cast the labels to floats, needed later
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))


# Create a TensorFlow optimizer, rather than using the Keras version
# This is currently necessary when working in eager mode
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

# We will now compile and print out a summary of our model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Step 1) Create a tf.data Dataset

Here, we'll use the `tf.data.Dataset` [API](https://www.tensorflow.org/api_docs/python/tf/data) to convert the Numpy arrays into a TensorFlow dataset.

We will create a simple for loop that will serve as our introduction into creating custom training loops. Although this essentially does the same thing as `model.fit` it allows us to customize the overall training process and collect different metrics throughout the process.

In [7]:
BATCH_SIZE=128

# Because tf.data may work with potentially **large** collections of data
# we do not shuffle the entire dataset by default
# Instead, we maintain a buffer of SHUFFLE_SIZE elements
# and sample from there.
SHUFFLE_SIZE = 10000 

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
dataset = dataset.shuffle(SHUFFLE_SIZE)
dataset = dataset.batch(BATCH_SIZE)

### Step 2) Iterate over the dataset
Here, we'll iterate over the dataset, and train our model using `model.train_on_batch`. To learn more about the elements returned from the dataset, you can print them out and try the `.numpy()` method.


In [8]:
EPOCHS=5

for epoch in range(EPOCHS):
  for images, labels in dataset:
    train_loss, train_accuracy = model.train_on_batch(images, labels)
  
  # Here you can gather any metrics or adjust your training parameters
  print('Epoch #%d\t Loss: %.6f\tAccuracy: %.6f' % (epoch + 1, train_loss, train_accuracy))
  

Epoch #1	 Loss: 0.163038	Accuracy: 0.927083
Epoch #2	 Loss: 0.140783	Accuracy: 0.958333
Epoch #3	 Loss: 0.013712	Accuracy: 0.989583
Epoch #4	 Loss: 0.048416	Accuracy: 0.989583
Epoch #5	 Loss: 0.009129	Accuracy: 1.000000


In [9]:
loss, accuracy = model.evaluate(test_images, test_labels)
print('Test accuracy: %.2f' % (accuracy))

10000/10000 [==============================] - 1s 88us/step
Test accuracy: 0.98


### Congratulations
You have trained a model on MNIST using Keras, eager execution, and tf.data.